In [1]:
from pynq import Overlay, allocate
import numpy as np

In [2]:
bitfile = "stream_matmul.bit"
overlay = Overlay(bitfile)
overlay.ip_dict.keys()

dict_keys(['blinkled_0/saxi'])

In [3]:
blinkled = overlay.blinkled_0

In [4]:
matrix_size = 16

a = allocate(shape=(matrix_size, matrix_size), dtype=np.int32)
b = allocate(shape=(matrix_size, matrix_size), dtype=np.int32)
c = allocate(shape=(matrix_size, matrix_size), dtype=np.int32)

a_addr = a.physical_address
b_addr = b.physical_address
c_addr = c.physical_address

In [5]:
a[:] = np.random.randint(-20, 20, (matrix_size, matrix_size), dtype=np.int32)
b[:] = np.random.randint(-20, 20, (matrix_size, matrix_size), dtype=np.int32).T
c[:] = np.zeros((matrix_size, matrix_size), dtype=np.int32)
print(c)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [6]:
# matrix_size, a_offset, b_offset, c_offset
blinkled.saxi.write(1 * 4, matrix_size)
blinkled.saxi.write(2 * 4, a_addr)
blinkled.saxi.write(3 * 4, b_addr)
blinkled.saxi.write(4 * 4, c_addr)

# start
blinkled.saxi.write(0 * 4, 1)

# busy wait
while True:
    busy = blinkled.saxi.read(5 * 4)
    if not busy:
        break

In [7]:
print(c)

[[ -313  -625  -221   527   361  -956  -382  -983  -406   -19   993   -54
    -80  -507  -261     4]
 [  159   108   924  -130  -512   857  -341  -349  -146     4   726  -242
    334   -58   345    16]
 [  -10  -112  -282   360  -439   289   642  -719   117   205  -118  -254
   -913   115  -122  -588]
 [  828  -389   902   652   114   166   307   166    60   153   831  -392
   -292   137  -532   578]
 [    5   819   137   143   836   198   -43  -296    70   -92   -68   443
   -479   275  -481  -738]
 [  415  1294  -331    50  -519   420  -580    -2  -200  -126  -519  -406
     87    95   339  -822]
 [  272  1037   729  -304  -856  1164  -124  -464  -495   -71   376  -112
    377   -62  -648  -293]
 [ -448    38  -546   188   199  -946   -47  -536  -441  -146  -898  -562
   -830   -46   374  -449]
 [  -31  -838  -335    -8  -317 -1030   217   170   -80   470  -334  -725
    222  -324  1393  1024]
 [  658   216   865  -235    -7  -622  -748 -1428  -276  -690   145  -869
    577   -40   8

In [8]:
expected = np.matmul(a, b.T)
print(expected)

[[ -313  -625  -221   527   361  -956  -382  -983  -406   -19   993   -54
    -80  -507  -261     4]
 [  159   108   924  -130  -512   857  -341  -349  -146     4   726  -242
    334   -58   345    16]
 [  -10  -112  -282   360  -439   289   642  -719   117   205  -118  -254
   -913   115  -122  -588]
 [  828  -389   902   652   114   166   307   166    60   153   831  -392
   -292   137  -532   578]
 [    5   819   137   143   836   198   -43  -296    70   -92   -68   443
   -479   275  -481  -738]
 [  415  1294  -331    50  -519   420  -580    -2  -200  -126  -519  -406
     87    95   339  -822]
 [  272  1037   729  -304  -856  1164  -124  -464  -495   -71   376  -112
    377   -62  -648  -293]
 [ -448    38  -546   188   199  -946   -47  -536  -441  -146  -898  -562
   -830   -46   374  -449]
 [  -31  -838  -335    -8  -317 -1030   217   170   -80   470  -334  -725
    222  -324  1393  1024]
 [  658   216   865  -235    -7  -622  -748 -1428  -276  -690   145  -869
    577   -40   8

In [9]:
diff_sum = np.sum(expected - c)
print(diff_sum)

0
